In [1]:
import tensorflow as tf
print(tf.__version__)

from tensorflow import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2.9.1


Примечание: в качестве дополнительного задания построения и обучения нейросети на новом датасете данные в методичках и ноутбуках разнятся. 

Где-то дается "два массива с рукописными буквами и с характеристиками вина.", или можно вернуться к обычному mnist.

А в блокноте с видеоурока - "Попробуйте обучить нейронную сеть на TensorFlow 2 на датасете imdb_reviews."

Я решил, в учебных целях, разницы особой нет. Взял последнее.

Задача нейросети тривиальна: обучиться на размеченных рецензиях на фильмы на сайте imdb и предсказывать эмоциональную окраску обзоров.

In [2]:
from keras.datasets import imdb

Описание датасета:

Датасет IMDb состоит из 50 000 обзоров фильмов от пользователей, помеченных как положительные (1) или отрицательные (0).

Рецензии предварительно обрабатываются, каждая из них кодируется последовательностью индексов слов в виде целых чисел.

Слова в обзорах индексируются по их общей частоте появления в датасете. Например, целое число «2» кодирует второе наиболее часто используемое слово.

Загружаем датасет.

In [3]:
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

В нем 50000 рецензий поделены по 25000: для обучения и для тестирования. 

Поэтому их соединю для последующего разделения 40000/10000.

In [4]:
data = np.concatenate((train_data, test_data), axis=0)

labels = np.concatenate((train_labels, test_labels), axis=0)

In [5]:
print("Категорий:", np.unique(labels))
print("Уникальных слов:", len(np.unique(np.hstack(data))))

Категорий: [0 1]
Уникальных слов: 9998


In [6]:
print("В среднем слов на рецензию:", round(np.mean([len(i) for i in data]), 1))
print("Стандартное отклонение:", round(np.std([len(i) for i in data]), 1))

В среднем слов на рецензию: 234.8
Стандартное отклонение: 172.9


В качестве примера посмотрим на случайную рецензию.

In [7]:
print(data[4])

[1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 14, 20, 56, 33, 2401, 18, 457, 88, 13, 2626, 1400, 45, 3171, 13, 70, 79, 49, 706, 919, 13, 16, 355, 340, 355, 1696, 96, 143, 4, 22, 32, 289, 7, 61, 369, 71, 2359, 5, 13, 16, 131, 2073, 249, 114, 249, 229, 249, 20, 13, 28, 126, 110, 13, 473, 8, 569, 61, 419, 56, 429, 6, 1513, 18, 35, 534, 95, 474, 570, 5, 25, 124, 138, 88, 12, 421, 1543, 52, 725, 6397, 61, 419, 11, 13, 1571, 15, 1543, 20, 11, 4, 2, 5, 296, 12, 3524, 5, 15, 421, 128, 74, 233, 334, 207, 126, 224, 12, 562, 298, 2167, 1272, 7, 2601, 5, 516, 988, 43, 8, 79, 120, 15, 595, 13, 784, 25, 3171, 18, 165, 170, 143, 19, 14, 5, 7224, 6, 226, 251, 7, 61, 113]


In [8]:
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()]) 
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[4]] )
print(decoded)

# worst mistake of my life br br i picked this movie up at target for 5 because i figured hey it's sandler i can get some cheap laughs i was wrong completely wrong mid way through the film all three of my friends were asleep and i was still suffering worst plot worst script worst movie i have ever seen i wanted to hit my head up against a wall for an hour then i'd stop and you know why because it felt damn good upon bashing my head in i stuck that damn movie in the # and watched it burn and that felt better than anything else i've ever done it took american psycho army of darkness and kill bill just to get over that crap i hate you sandler for actually going through with this and ruining a whole day of my life


Похоже, негативная рецензия.

ПОДГОТОВКА ДАННЫХ

Самая большая рецензия содержит максимум 10000 слов. Нужно векторизовать объекты так, чтобы имели одинаковую длину.

In [9]:
def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results
 
    
data = tf.convert_to_tensor(vectorize(data), np.float32) 
labels = tf.convert_to_tensor(labels, np.float32) 

#data = vectorize(data)
#labels = np.array(labels).astype("float32")

Деление на тест и трейн

In [10]:
test_x = tf.convert_to_tensor(data[:10000])
test_y = tf.convert_to_tensor(labels[:10000])
train_x = tf.convert_to_tensor(data[10000:])
train_y = tf.convert_to_tensor(labels[10000:])

ПОСТРОЕНИЕ МОДЕЛИ

In [46]:
model = keras.Sequential([
    keras.layers.Dense(32, activation = "elu", input_shape=(10000, )),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(16, activation = "elu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation = "sigmoid")
])

In [47]:
model.compile(
 optimizer = "adam",
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)

In [48]:
model.fit(train_x, train_y, 
          batch_size = 64, 
          epochs=3
         )

Epoch 1/3
625/625 [==============================] - 7s 9ms/step - loss: 0.3480 - accuracy: 0.8551
Epoch 2/3
625/625 [==============================] - 6s 9ms/step - loss: 0.2429 - accuracy: 0.9085
Epoch 3/3
625/625 [==============================] - 6s 9ms/step - loss: 0.2101 - accuracy: 0.9213


In [49]:
test_loss, test_acc = model.evaluate(test_x,  test_y, verbose=2)

print('\nTest accuracy:', round(test_acc, 4))

313/313 - 2s - loss: 0.2874 - accuracy: 0.8879 - 2s/epoch - 5ms/step

Test accuracy: 0.8879


Итак, нейросеть по содержанию (наличию и частоте встречаемости слов) предсказывает эмоциональную окраску рецензий с точностью $~0.89$

Модель склонна переобучаться. Довел разницу между трейном и тестом до минимальных 3.8% на данных параметрах.

P.S. Чтобы было честно:

отталкивался от поста на хабре (https://habr.com/ru/post/477630), который подсказал, как читать датасет imdb_reviews и векторизовать данные.

